In [57]:
import spacy as spacy
import json 
from main import pdfDocToStringEn, fixRandomSubstitutions
from spacy.matcher import Matcher 
from spacy.tokens import Span#, DocBin 

In [74]:
nlp = spacy.blank('en')

fName = r"F:\Python\OCR_diploma\docs_base\StauffInvoices\92922860.txt"
fName = r"F:\Python\OCR_diploma\docs_base\StauffInvoices\92946889.pdf"
fName = r"C:\Users\alex1c\sources\spacySt\OCR\0093012934.pdf"
fName = r"C:\Users\alex1c\sources\spacySt\OCR\0093012939.pdf"
fName = r"C:\Users\alex1c\sources\spacySt\OCR\93016136.pdf"
strSeparator = '\n#########################################################\n'
fTextName = fName.replace('.pdf', '.txt')
print(fTextName)

In [75]:
text = pdfDocToStringEn(fName)
with open(fTextName, encoding="utf8", mode='w') as f:
    f.write(text)

In [76]:
print(text)

CA ®
STAUFF Deutschland amaS TAUFE

Invoice
Walter Stauffenberg GmbH & Co. KG, Im Ehrenfeld 4, D-58791 Werdohl

STAUFF L.L.C.

Str Zarechnaya, House 153 korpus 4 Number: 93016136
Date: 12.05.2022
shchelkovo Customer no.: 95015
141100 MOSKOVSKAYA OBLAST ”
RUSSISCHE FOED. Your VAT-ID-no.:
Contact CSC: Asuncion Perez Martinez
Phone: 02392/916 -163
E-Mail: a.perez@stauff.com
Contact sales: Martin Meijk
Phone: 02392/916 -151
Forwarder: E-Mail: m.meijk@stauff.com

MBI International Spedition GmbH, Bielefeld

Your fax-no.: 007 4952761651
Your phone-no.: 007 4952761650
Your contact person: Ms. Bakanova

Shipment:
- By Truck -

Delivery address:

STAUFF L.L.C. *

= > Project: RSM

Str Zarechnaya, House 153 korpus 4
Shchelkovo

141100 MOSKOVSKAYA OBLAST
RUSSISCHE FOED.

Beneficiary:
STAUFF L.L.C.

Str Zarechnaya, House 153 korpus 4
Shchelkovo

141100 MOSKOVSKAYA OBLAST
RUSSISCHE FOED.

Please note that we can only guarantee the stated prices for deliveries made before October 12th, 2021. Due to 

In [3]:
with open(fTextName, encoding="utf8") as f:
    text = f.read()

In [52]:
#unused
ttxt = "Qty/Unit Article code Price EUR Unit Amount EUR \n Description" #.replace('\n', '')
docDesc = nlp(fixRandomSubstitutions(ttxt))
#for w in docDesc:  print(w)
pattern4_1 = [
    {'LOWER':'article'},
    {'LOWER':'code'}, 
    {'LOWER':'price'}, 
    {'LOWER':'eur'},
    {'LOWER':'unit'},
    {'LOWER':'amount'},
    {'LOWER':'eur'},
    {'IS_ALPHA': False, 'OP' : '?'},
    {'LOWER':'description'}
    ]
matcher = Matcher(nlp.vocab)
matcher.add('STARTPAGE', [pattern4_1])
matches = matcher(docDesc)
spans = [Span(docDesc, start, end, label=match_id) for match_id, start, end in matches]
for span in spans:
    print(span.text)

Article code Price EUR Unit Amount EUR 
 Description


In [60]:
def shortening(text, verbose = True):
    nlp = spacy.blank("en")

    doc = nlp(text)

    matcher = Matcher(nlp.vocab)

    pattern3 = [
        {'LOWER':'CONTRIBUTING'.lower()},
        {'LOWER':'TO'.lower()}, 
        {'LOWER':'YOUR'.lower()}, 
        {'LOWER':'SUCCESS'.lower()}
        ]
    matcher.add('ENDPAGE', [pattern3])

    pattern4 = [
        {'LOWER':'article'},
        {'LOWER':'code'}, 
        {'LOWER':'price'}, 
        {'LOWER':'eur'},
        {'LOWER':'unit'},
        {'LOWER':'amount'},
        {'LOWER':'eur'},
        {'TEXT':'\n', 'OP' : '?'},
        {'LOWER':'description'}
    ]
        #Article code Price EUR Unit Amount EUR Description
    pattern4_1 = [
        {'LOWER':'article'},
        {'LOWER':'code'}, 
        {'LOWER':'price'}, 
        {'LOWER':'eur'},
        {'LOWER':'unit'},
        {'LOWER':'amount'},
        {'LOWER':'eur'},
        {'IS_ALPHA': False, 'OP' : '?'},
        {'LOWER':'description'}
    ]

    pattern4_2 = [{'LOWER':'russische'}, {'LOWER':'foed'}]
    matcher.add('STARTPAGE', [pattern4, pattern4_1, pattern4_2])
    matcher.add('STARTPAGE', [pattern4_1])

    pattern5  = [
        {'LOWER':'doc.no'},
        {'LOWER':'.'},
        {'LOWER':'/'},
        {'LOWER':'date'},
        {'LOWER':'page'}
    ]
    
    pattern6  = [
        {'LOWER':'doc.по'},
        {'LOWER':'.'},
        {'LOWER':'/'},
        {'LOWER':'date'},
        {'LOWER':'page'}
    ] 
    matcher.add('ABOUTNOPAGE', [pattern5, pattern6])

    doc = nlp(text)

    matches = matcher(doc)
    spans = [Span(doc, start, end, label=match_id) for match_id, start, end in matches]

    paging = []

    pageinfo = {'ABOUTNOPAGE':0, 'STARTPAGE':0, 'ENDPAGE':0}
    shortenText = ''
    firstPage = True
    txt = '''import spacy
    from spacy.tokens import Span

    nlp = spacy.blank("en")\n'''
    docs  = []
    npage = 0
    for sp in spans:
        if sp.label_  == 'ABOUTNOPAGE':
            pageinfo[sp.label_] = sp.end
        elif sp.label_  == 'STARTPAGE':
            pageinfo[sp.label_] = sp.end
            #print(sp.text)
        elif sp.label_  == 'ENDPAGE':
            pageinfo[sp.label_] = sp.start
            paging.append(pageinfo)
            
            if firstPage ==True:
                shortenText = doc[:pageinfo['ENDPAGE']].text
                firstPage = False
                docs.append(doc[:pageinfo['ENDPAGE']].text)
                txt = txt + strSeparator + doc[:pageinfo['ENDPAGE']].text 
            else:
                shortenText = shortenText +  doc[pageinfo['STARTPAGE']:pageinfo['ENDPAGE']].text
                docs.append(doc[pageinfo['STARTPAGE']:pageinfo['ENDPAGE']].text)
                if verbose: print(pageinfo, pageinfo['ENDPAGE'] - pageinfo['STARTPAGE'])
                txt = txt + strSeparator + doc[pageinfo['STARTPAGE']:pageinfo['ENDPAGE']].text 

                #print(doc[pageinfo['STARTPAGE']:pageinfo['ENDPAGE']].text)
                #break
            pageinfo = {'ABOUTNOPAGE':0, 'STARTPAGE':0, 'ENDPAGE':0}
            npage += 1

        #print(pageinfo)
    #print(txt)
    if verbose: print(len(paging))
    return txt

In [61]:
txt = shortening(fixRandomSubstitutions(text), verbose=False)
txt = txt.replace('\n\n',' ').replace('  ', ' ')
#print(txt.replace('\n\n',' ').replace('  ', ' '))
with open(fTextName.replace('.txt', 's.txt'), encoding="utf8", mode='w') as f:
    f.write(txt)

In [20]:
#unused
with open(fTextName.replace('.txt', 's.txt'), encoding="utf8") as f:     
    rtest = f.read()
nlp = spacy.blank("en")
rtest = rtest.replace(strSeparator, '').replace('\n\n', ' ').replace('  ', ' ')
with open(fTextName.replace('.txt', 'ls.txt'), encoding="utf8", mode='w') as f:     
    #rtest = f.read()
    f.write(rtest)
exit
doc1 = nlp(rtest.replace(strSeparator, ''))

print(len(doc1))
col = 10
cc = col
lst = []
row = []
nrow = 0
txt = ''
for d in doc1:
    lst.append(d)
    txt += d.text + '\n'
    continue
    cc -=1
    row.append(d)
    if cc == 0:
        lst.append(row)
        row = []
        cc = col
        nrow +=1
        #break
    
if not len(row) == 0:
    lst.append(row)
print(nrow)
        
len(lst[0])
with open(fTextName.replace('.txt', 'ls.txt'), encoding="utf8", mode='w') as f:     
    #rtest = f.read()
    f.write(txt)



17542
0


In [62]:
def fixRandomSubstitutions(text):
    text = text.replace('А', 'A').replace('В', 'B').replace('С', 'C').replace('Е', 'E').replace('Н', 'H')
    text = text.replace('К', 'K').replace('М', 'M').replace('О', 'O').replace('Р', 'P').replace('Т', 'T').replace('Х', 'X')
    text = text.replace('а', 'a').replace('с', 'c').replace('е', 'e').replace('о', 'o').replace('р', 'p')
    text = text.replace('п', 'n').replace('х', 'x')
    return text


In [6]:
fixRandomSubstitutions('АСКОНА')

'ACKOHA'

In [ ]:
lst

In [ ]:
rtest = '''
Order number: 2378632
Purchase order number: N SR-1-06 1772 2400 25 PC 1120002537 38,09 100 PC 9,52
SP-2D-M-W2
SP 2 DM W2 packed per each item Product description: weld plate Export - Customs tariff no.: 73269098
Country of origin: Germany Material Surcharge 11,40 % 1,09
10,61
Order number: 2380831
Purchase order number: N SR-1-06 1808
2500 150 PC 1020023990 42,38 1 PC 6.357 ,00 SN-045-E-20-B/4 SN-045E20B/4 packed per each item Product description: filter element
Export - Customs tariff no.: 84219990
Country of origin: Germany Material Surcharge 8,90 % 565,77
6.922,77
EAC - Eurasian Conformity
'''

rtest = '''Order number: 2390568
Purchase order number: N SR-1-06 1911** 7300 75 PC 6030005626 (*) 157,47 100 PC 118,10
FI-SKM-35L-W3 packed per each item Product description: nuts Export - Customs tariff no.: 73181699
Country of origin: Germany Material Surcharge 6,90 % 8,15
Energy Surcharge 3,50 % 4,13
130,38'''
#with open(fName.replace('.txt', 's.txt'), encoding="utf8") as f:     
#   rtest = f.read()

nlp = spacy.blank("en")
matcher = Matcher(nlp.vocab)
patternTariff = [{'LOWER': 'customs'}, 
    {'LOWER':'tariff'}, 
    {'IS_ALPHA': True},
    {'IS_PUNCT': True, 'OP': '*'}, 
    #{'LOWER': '.'},
    #{'LOWER': ':'}, 
    {'IS_DIGIT': True}]
matcher.add('PTRTARIFF', [patternTariff])

patternPC = [{'LIKE_NUM': True}, {'LIKE_NUM': True},
    {'LOWER' : {'in' : ['pc', 'рс']}},
    {'LIKE_NUM':True}    ] 

matcher.add('PTROFPC', [patternPC])

doc1 = nlp(rtest)
ldoc = []
n  = 10
for d in doc1:
    if n == 10:

print("11:", [token.text for token in doc1[5:24]])
matches = matcher(doc1)
spans = [Span(doc1, start, end, label=match_id) for match_id, start, end in matches]
maxcol = 0
curcol = 0
for sp in doc1:
    #print(sp.text)
    curcol +=1
    if '\n' in sp.text:
        maxcol = max(maxcol, curcol) 
        curcol = 0
    

print(maxcol)

In [8]:
rtest = '''Order number: 2390568
Purchase order number: N SR-1-06 1911** 7300 75 PC 6030005626 (*) 157,47 100 PC 118,10
FI-SKM-35L-W3 packed per each item Product description: nuts Export - Customs tariff no.: 73181699
Country of origin: Germany Material Surcharge 6,90 % 8,15
Energy Surcharge 3,50 % 4,13
130,38'''

In [63]:
# add material + energy surcharge (16/05/22)

#with open(fName.replace('.txt', 's.txt'), encoding="utf8") as f: rtest = f.read()
rtest = rtest.replace(strSeparator, '')

nlp = spacy.blank("en")
matcher = Matcher(nlp.vocab)

patternNOrder = [{'LOWER': 'order'}, {'LOWER':'number'}, {'LOWER':':'}, {'LIKE_NUM': True}]
matcher.add('PTRNORDER', [patternNOrder])

patternPurchase = [{'LOWER': 'purchase'}, {'LOWER':'order'}, {'LOWER':'number'}, {'LOWER': ':'}]
matcher.add('PTRPURCHASE', [patternPurchase])

patternTariff = [{'LOWER': 'customs'}, {'LOWER':'tariff'}, {'IS_ALPHA': True}, {'IS_PUNCT': True, 'OP': '*'}, 
    {'IS_DIGIT': True}]
matcher.add('PTRTARIFF', [patternTariff])

patternCountry = [{'LOWER': 'country'}, {'LOWER':'of'}, {'LOWER': 'origin'}, {'LOWER': ':'}, {'IS_ALPHA': True}]
matcher.add('PTRCOUNTRY', [patternCountry])

patternPC = [{'LIKE_NUM': True}, {'LIKE_NUM': True}, {'LOWER' : {'in' : ['pc', 'рс']}}, {'LIKE_NUM':True}] 
matcher.add('PTROFPC', [patternPC])

patternSurcharge = [{'LOWER': 'material'}, {'LOWER': 'surcharge'}, {'LIKE_NUM' : True}] #, {'TEXT': '%'}, {'LIKE_NUM' : True}] 
matcher.add('PTRSURCH', [patternSurcharge])

patternESurcharge = [{'LOWER': 'energy'}, {'LOWER': 'surcharge'}, {'LIKE_NUM' : True} , {'TEXT': '%'}, {'LIKE_NUM' : True}] 
matcher.add('PTR_E_SURCH', [patternESurcharge])

doc1 = nlp(rtest)

matches = matcher(doc1)
spans = [Span(doc1, start, end, label=match_id) for match_id, start, end in matches]
print(len(doc1), len(spans))
startSpan = 0
endSpan = 0
posTariff = 0
txt2 = 'import json \nimport spacy \nfrom spacy.matcher import Matcher \nfrom spacy.tokens import Span, DocBin \ndocs = [] \n'
nPos = 0
firstPos = True
pos1PC = 0
pos2PC = 0
res = dict()

for sp in spans:
    if sp.label_ == 'PTRNORDER':
        res['NORDER'] = sp.start + 3
    #print(sp.start, sp.label_)
print(res)
exit
for sp in spans:
    if sp.label_  == 'PTRPURCHASE':
        startSpan = sp.start
        #print('here is a purchase', sp.start, doc1[sp.start: sp.end]) 
    elif sp.label_  == 'PTRTARIFF':
        posTariff = sp.end-1
        #print('here is a tariff', sp.end, doc1[sp.end-1: sp.end]) 
    elif sp.label_  == 'PTROFPC':
        if firstPos:
            print('here is an article', sp.end, doc1[sp.start: sp.end]) 
            posArtc =  sp.start + 3
            print('Article', doc1[posArtc])
            firstPos = False
        else:
            pos2PC = sp.start
            print('here is a summa', sp.end, doc1[sp.start: sp.end]) 
    elif sp.label_  == 'PTRCOUNTRY':
        #print('here is a country', sp.end, doc1[sp.start: sp.end]) 
        stPosCountry =  sp.start + 3
    elif sp.label_  == 'PTRSURCH':
        #print('here is a surcharge', sp.end, doc1[sp.start: sp.end]) 
        #                 pos  amo article   pr     uni  summe tariff   country
        #doc1 SR-1-06 711 1300 50 6010003476 237,81 100 118,91 73079910 Germany
        enPosCountry =  sp.start - 1
        posCharge =  sp.start + 2
        posChargeS =  sp.start + 4
    elif sp.label_  == 'PTR_E_SURCH':
        print('here is an energy surcharge', sp.end, doc1[sp.start: sp.end]) 
        #                 pos  amo article   pr     uni  summe tariff   country
        #doc1 SR-1-06 711 1300 50 6010003476 237,81 100 118,91 73079910 Germany
        
        posECharge =  sp.start + 2
        posEChargeS =  sp.start + 4
        posSum =  sp.start + 6
        endSpan = sp.end
        pos2PC = pos2PC - startSpan  

        print(nPos, startSpan, posArtc, stPosCountry, enPosCountry, posCharge, posChargeS, posECharge, posEChargeS, posSum, endSpan)

        print(doc1[startSpan: endSpan]) #, doc1[startSpan + pos2PC +1 ],doc1[startSpan + pos2PC+2],doc1[startSpan + pos2PC+3],            doc1[startSpan + pos2PC+4])  
        #break
        #print(doc1[startSpan+pos2PC + 4].text )
 
        sDoc = "doc" + str(nPos)
        txt2 = txt2 + '\n\n' + sDoc + " = nlp('''" + doc1[startSpan:endSpan].text + "''')\n" #        txt2 = ''
        txt2 = txt2 +  ('#' + sDoc  + ' c:' + doc1[startSpan + 5 : startSpan + 8].text + ' c1:' + doc1[startSpan + 8].text 
            + ' pos:' + doc1[startSpan + 10].text + ' a:' + doc1[startSpan + 11].text + ' art:' + doc1[posArtc].text 
            + ' pr:' + doc1[pos2PC].text + ' u:' + doc1[posArtc + 2].text + ' s:'  + doc1[posArtc + 4].text 
            + ' trf:' +  doc1[posTariff].text + ' cnt:' + doc1[stPosCountry: enPosCountry].text 
            + ' chr:' + doc1[posCharge].text + ' chrS:' + doc1[posChargeS].text  
            + ' chrE:' + doc1[posECharge].text + ' chrES:' + doc1[posEChargeS].text 
            + ' TTL:' + doc1[posSum].text).replace('\n', '')+ '\n'
        #txt2 = txt2.replace('\n', '') + '\n'

        txt2 = (txt2 + '''print(''' + '"' + sDoc + '",' + sDoc + "[5: 8], " + sDoc + "[8: 9], " + sDoc + "[10: 11], " 
            + sDoc +"[11: 12], " + sDoc + "[13: 14], " + sDoc + "["+str(pos2PC) + ':' + str(pos2PC + 1) + "], "
            + sDoc + "[" +str(pos2PC+1) + ':' + str(pos2PC + 2) + "] ," + sDoc + "["+str(pos2PC + 3) + ':' + str(pos2PC + 4) + "], " 
            + sDoc + "["+ str(posTariff )+ ': ' + str(posTariff + 1) + "], " + sDoc + "[" + str(stPosCountry)+ ': ' + str(enPosCountry) + "]), "
            + sDoc + "["+ str(posCharge )+ ': ' + str(posCharge + 1) + "], " 
            + sDoc + "["+ str(posChargeS )+ ': ' + str(posChargeS + 1) + "], " 
            + sDoc + "["+ str(posECharge )+ ': ' + str(posECharge + 1) + "], " 
            + sDoc + "["+ str(posEChargeS )+ ': ' + str(posEChargeS + 1) + "], " 
            + sDoc + "["+ str(posSum )+ ': ' + str(posSum + 1) + "]  \n")
        txt2 = txt2 +  sDoc + ".ents = [Span(" + sDoc + ', 5, 8, label="CONTRACT"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', 8, 9, label="CONTRACT1"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', 10, 11, label="POS"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', 11, 12, label="AMOUNT"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', '+ str(posArtc) + ', ' + str(posArtc +1) +', label="ARTICLE"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(pos2PC) + ', ' + str(pos2PC + 1) + ', label="PRICE"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(pos2PC + 1) + ', ' + str(pos2PC + 2) + ', label="UNIT"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(pos2PC + 3) + ', ' + str(pos2PC + 4) + ', label="SUM"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(posTariff)+ ', ' + str(posTariff + 1) + ', label="TARIFF"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(stPosCountry)+ ', ' + str(enPosCountry) + ', label="COUNTRY"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(posCharge)+ ', ' + str(posCharge + 1) + ', label="CHARGE"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(posChargeS)+ ', ' + str(posChargeS + 1) + ', label="CHARGES"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(posECharge)+ ', ' + str(posECharge + 1) + ', label="ECHARGE"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(posEChargeS)+ ', ' + str(posEChargeS + 1) + ', label="ECHARGES"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(posSum)+ ', ' + str(posSum + 1) + ', label="TOTAL") \n'
        txt2 = txt2 + 'docs.append(' + sDoc + ')\n'
        nPos += 1
        break
print(txt2)
exit

txt2 = txt2 + 'doc_bin = DocBin(docs = docs)\n'
txt2 = txt2 + "doc_bin.to_disk('./train.spacy')"
with open(fName.replace('.txt', '.py'), encoding="utf8", mode='w') as f:
    f.write(txt2)

75 7
{'NORDER': 4}
here is an article 30 251,01 100 PC 32,63
Article 32,63
here is an energy surcharge 72 Energy Surcharge 3,50 % 1,14
0 6 29 59 60 63 65 69 71 73 72
Purchase order number: N SR-1-06 1911**
17300 13. PC 6020000130 (*) 251,01 100 PC 32,63 FI-WS-15L-W3-OGR-SKM packed per each item Product description: fitting Export - Customs tariff no.: 73079290
Country of origin: Germany Material Surcharge 6,90 % 2,25
Energy Surcharge 3,50 % 1,14
import json 
import spacy 
from spacy.matcher import Matcher 
from spacy.tokens import Span, DocBin 
docs = [] 


doc0 = nlp('''Purchase order number: N SR-1-06 1911**
17300 13. PC 6020000130 (*) 251,01 100 PC 32,63 FI-WS-15L-W3-OGR-SKM packed per each item Product description: fitting Export - Customs tariff no.: 73079290
Country of origin: Germany Material Surcharge 6,90 % 2,25
Energy Surcharge 3,50 % 1,14''')
#doc0 c:SR-1-06 c1:1911 pos:* a: art:32,63 pr:3,50 u:- s:- trf:73079290 cnt:: chr:6,90 chrS:2,25 chrE:3,50 chrES:1,14 TTL:36,02
print(

In [40]:
dd = {'99':99}
dd['1'] = 1
dd.pop('2', 2)
print(dd)

{'99': 99, '1': 1}


# add material surcharge - advanced parcing

In [64]:
def partCodePrepare(doc1, res, pos):
    sDoc = 'doc' +str(pos)
    codeCom = '#'
    #code = sDoc+'.ents = [\n'
    firstPos = 100000
    lastPos = 0
    codeMain = ''
    for key in res.keys():
        #lastPos = max(res.get(key)[1], lastPos)
        #firstPos = min(firstPos, res.get(key)[0])
        codeCom += key + ' ' + res.get(key)[2].text + '; '
        if codeMain !='':
            codeMain += (',\n   Span(' + sDoc + ', ' + str(res.get(key)[0]) 
                + ', ' + str(res.get(key)[1]) + ', label ="' + key + '")')
        else:
            codeMain += ('   Span(' + sDoc + ', ' + str(res.get(key)[0]) 
                + ', ' + str(res.get(key)[1]) + ', label ="' + key + '")')
    code = ('print("' + sDoc + ', ' + str(len(doc1)) + ', '+ codeCom + '")\n' + sDoc 
        + " = nlp('''" + doc1.text + "''')\n")
    code += sDoc+'.ents = [\n' + codeMain + ']\ndocs.append(' + sDoc + ')\n'
    #print(codeCom, firstPos, lastPos)
    return code

In [65]:
def parceSomeText(rtest, pos):
    doc1 = nlp(rtest.replace('. ', ' '))
    #print(type(doc1), len(doc1), doc1[-1])
    matcher = Matcher(nlp.vocab)

    patternNOrder = [{'LOWER': 'order'}, {'LOWER':'number'}, {'LOWER':':'}, {'LIKE_NUM': True}]
    matcher.add('NORDER', [patternNOrder])
    matches = matcher(doc1)

    patternPurchase = [{'LOWER': 'purchase'}, {'LOWER':'order'}, {'LOWER':'number'}, {'LOWER': ':'}]
    matcher.add('PURCHASEORDER', [patternPurchase])

    patternTariff = [{'LOWER': 'customs'}, {'LOWER':'tariff'}, {'IS_ALPHA': True}, {'IS_PUNCT': True, 'OP': '*'}, {'IS_DIGIT': True}]
    matcher.add('TARIFF', [patternTariff])

    patternCountry = [{'LOWER': 'country'}, {'LOWER':'of'}, {'LOWER': 'origin'}, {'LOWER': ':'}, {'IS_ALPHA': True}]
    matcher.add('COUNTRY', [patternCountry])

    patternPC = [{'LIKE_NUM': True}, {'IS_PUNCT': True, 'OP': '*'}, {'LIKE_NUM': True}
        , {'LOWER' : {'in' : ['pc', 'рс']}}, {'LIKE_NUM':True}
        , {'IS_PUNCT':True, 'OP':'?'}] 
    matcher.add('POSPC', [patternPC])

    patternSurcharge = [{'LOWER': 'material'}, {'LOWER': 'surcharge'}, {'LIKE_NUM' : True}] #, {'TEXT': '%'}, {'LIKE_NUM' : True}] 
    matcher.add('SURCHARGE', [patternSurcharge])
    matches = matcher(doc1)

    patternESurcharge = [{'LOWER': 'energy'}, {'LOWER': 'surcharge'}, {'LIKE_NUM' : True}] #, {'TEXT': '%'}, {'LIKE_NUM' : True}] 
    matcher.add('ESURCHARGE', [patternESurcharge])
    matches = matcher(doc1)

    spans = [Span(doc1, start, end, label=match_id) for match_id, start, end in matches]

    matcher2 = Matcher(nlp.vocab)
    patternNums = [{'LIKE_NUM':True}]
    matcher2.add('NUMBERS', [patternNums])

    res = None

    code = ''
    #pos = 0
    for sp in spans:
        #print(sp.start, sp.end, sp.text)
        if sp.label_ == 'NORDER':
            if res != None: 
                code += partCodePrepare(res, pos)
                #pos += 1
            res = {'NORDER' : (sp.start + 3, sp.start + 4, doc1[sp.start + 3: sp.start + 4])}
            firstPC = True
        if sp.label_ == 'PURCHASEORDER':
            if res==None: res = {}
            res['CONTRACT'] = (sp.start + 5, sp.start + 8, doc1[sp.start + 5: sp.start + 8])
            res['CONTRACT1'] = (sp.start + 8, sp.start + 9, doc1[sp.start + 8: sp.start + 9])
        if sp.label_ == 'POSPC' and firstPC:
            print("span - POS", doc1[sp.start: sp.end])
            res['POS'] = (sp.start , sp.start + 1, doc1[sp.start: sp.start + 1])
            res['AMOUNT'] = (sp.start + 1, sp.start + 2, doc1[sp.start + 1: sp.start + 2])
            res['ARTICLE'] = (sp.start + 3, sp.start + 4, doc1[sp.start + 3: sp.start + 4])
            firstPC = False
        if sp.label_ == 'POSPC' and not firstPC:
            pDoc = sp.text
            doc3 = nlp(pDoc)
            matches2 = matcher2(doc3)
            spanN = [Span(doc3, start, end, label=match_id) for match_id, start, end in matches2]
            spP, spU, spS = spanN[0].start, spanN[1].start, spanN[2].start
            res['PRICE'] = (sp.start + spP , sp.start + spP + 1, doc1[sp.start + spP: sp.start + spP + 1])
            res['UNIT'] = (sp.start + spU, sp.start + spU + 1, doc1[sp.start + spU: sp.start + spU + 1])
            res['SUMMA'] = (sp.start + spS, sp.start + spS + 1, doc1[sp.start + spS: sp.start + spS + 1])
            #break
        if sp.label_ == 'TARIFF':
            res['TARIFF'] = (sp.end -1 , sp.end, doc1[sp.end - 1: sp.end])  
        if sp.label_ == 'COUNTRY':
            #print(doc1[sp.end].text.lower())
            if doc1[sp.end].text.lower() == 'material':
                res['COUNTRY'] = (sp.end -1 , sp.end, doc1[sp.end - 1: sp.end])    
            else:    
                res['COUNTRY'] = (sp.end -1 , sp.end + 1, doc1[sp.end - 1: sp.end + 1]) 
        if sp.label_ == 'SURCHARGE':
            #print(sp.start, len(doc1), doc1[sp.start:])
            pDoc = doc1[sp.start:].text
            doc3 = nlp(pDoc)
            matches2 = matcher2(doc3)
            spanN = [Span(doc3, start, end, label=match_id) for match_id, start, end in matches2]
            #print(pDoc, spanN)
            spP, spS, spT = spanN[0].start, spanN[1].start, spanN[2].start
            #print(spP, spS, spT)
            res['PR_SURCH'] = (sp.start + spP, sp.start + spP + 1, doc1[sp.start + spP: sp.start + spP + 1])
            res['SURCHARGE'] = (sp.start + spS, sp.start + spS + 1, doc1[sp.start + spS: sp.start + spS +1])
        if sp.label_ == 'ESURCHARGE':
            #print(sp.start, len(doc1), doc1[sp.start:])
            pDoc = doc1[sp.start:].text
            doc3 = nlp(pDoc)
            matches2 = matcher2(doc3)
            spanN = [Span(doc3, start, end, label=match_id) for match_id, start, end in matches2]
            #print(pDoc, spanN)
            spP, spS, spT = spanN[0].start, spanN[1].start, spanN[2].start
            #print(spP, spS, spT)
            res['PR_ESURCH'] = (sp.start + spP, sp.start + spP + 1, doc1[sp.start + spP: sp.start + spP + 1])
            res['ESURCHARGE'] = (sp.start + spS, sp.start + spS + 1, doc1[sp.start + spS: sp.start + spS +1])
            res['TOTAL'] = (sp.start + spT, sp.start + spT + 1, doc1[sp.start + spT: sp.start + spT + 1])
            if res != None: 
                code += partCodePrepare(doc1, res, pos)
                #pos += 1
                res = None
    return code


In [51]:
rtest = '''
Order number: 2390568
Purchase order number: N SR-1-06 1911**
17300 13. PC 6020000130 (*) 251,01 100 PC 32,63 FI-WS-15L-W3-OGR-SKM packed per each item Product description: fitting Export - Customs tariff no.: 73079290
Country of origin: Germany Material Surcharge 6,90 % 2,25
Energy Surcharge 3,50 % 1,14
36,02

'''

In [36]:
doc1

'Order number: 2332617 Purchase order number: N SR-1-06 1231 2000 2 PC 6010003898 (* ) 102,90 100 PC 2,06 FI-REDSD-1 0/08L-V-W3-DKO FI-REDSD-1 0/08L-B-W3-DKO packed per each item Product description: fitting Export - Customs tariff no.: 73079910 Country of origin: Germany Material Surcharge 6,90 % 0,14    2,20'

In [44]:
print(parceSomeText(rtest, 1))

span - POS 17300 13 PC 6020000130
print("doc1, 74, #NORDER 2390568; CONTRACT SR-1-06; CONTRACT1 1911; POS 17300; AMOUNT 13; ARTICLE 6020000130; PRICE 251,01; UNIT 100; SUMMA 32,63; TARIFF 73079290; COUNTRY Germany; PR_SURCH 6,90; SURCHARGE 2,25; PR_ESURCH 3,50; ESURCHARGE 1,14; TOTAL 36,02; ")
doc1 = nlp('''
Order number: 2390568
Purchase order number: N SR-1-06 1911**
17300 13 PC 6020000130 (*) 251,01 100 PC 32,63 FI-WS-15L-W3-OGR-SKM packed per each item Product description: fitting Export - Customs tariff no.: 73079290
Country of origin: Germany Material Surcharge 6,90 % 2,25
Energy Surcharge 3,50 % 1,14
36,02

''')
doc1.ents = [
   Span(doc1, 4, 5, label ="NORDER"),
   Span(doc1, 11, 14, label ="CONTRACT"),
   Span(doc1, 14, 15, label ="CONTRACT1"),
   Span(doc1, 18, 19, label ="POS"),
   Span(doc1, 19, 20, label ="AMOUNT"),
   Span(doc1, 21, 22, label ="ARTICLE"),
   Span(doc1, 25, 26, label ="PRICE"),
   Span(doc1, 26, 27, label ="UNIT"),
   Span(doc1, 28, 29, label ="SUMMA"),
  

In [66]:
with open(fTextName.replace('.txt', 's.txt'), encoding="utf8") as f: 
    rtest = f.read()
rtest = rtest.replace(strSeparator.replace('\n',''), '').replace('\n', ' ')

nlp = spacy.blank("en")
fullDoc = nlp(rtest)
matcher = Matcher(nlp.vocab)

patternNOrder = [{'LOWER': 'order'}, {'LOWER':'number'}, {'LOWER':':'}, {'LIKE_NUM': True}]
matcher.add('NORDER', [patternNOrder])
matches = matcher(fullDoc)
prepSpans = [Span(fullDoc, start, end, label=match_id) for match_id, start, end in matches]

#txt2 = 'import json \nimport spacy \nfrom spacy.matcher import Matcher \nfrom spacy.tokens import Span, DocBin \ndocs = [] \n'
stPos = 0
code = ''
pos = 0
for prepItem in prepSpans:
    #print(prepItem.start)
    if stPos == 0:
        stPos = prepItem.start
    else:
        doc1 = fullDoc[stPos : prepItem.start].text
        code1 = parceSomeText(doc1, pos)
       
        pos += 1

        #print(code1)
        code +=code1
        stPos = prepItem.start 
                            

txt2 = '''import json 
import spacy 
from spacy.matcher import Matcher 
from spacy.tokens import Span, DocBin 
docs = []\n''' + code + 'doc_bin = DocBin(docs = docs)\n'
txt2 = txt2 + "doc_bin.to_disk('./train1.spacy')"
with open(fTextName.replace('.txt', '.py'), encoding="utf8", mode='w') as f:
    f.write(txt2)

print('Selected ', pos, 'examples')

span - POS 100 600 PC 1730002340
span - POS 200 6.250 PC 1130022786
span - POS 300 10.000 PC 1130022886
span - POS 400 500 PC 1130022787
span - POS 500 750 PC 1130024670
span - POS 600 600 PC 1210026027
span - POS 700 200 PC 1210026041
span - POS 900 125 PC 1130000261
span - POS 1000 100 PC 1130000264
span - POS 1100 700 PC 1130000267
span - POS 1200 1.000 PC 1130000273
span - POS 1300 2.700 PC 1130001959
span - POS 1400 300 PC 1130001960
span - POS 1500 700 PC 1130001961
span - POS 1600 50 PC 1130000764
span - POS 1700 50 PC 1130000765
span - POS 1800 400 PC 1130000766
span - POS 1900 1.000 PC 1130000767
span - POS 2000 12.800 PC 1130022187
span - POS 2100 2.647 PC 1130022183
span - POS 2200 1.000 PC 1130022185
span - POS 2300 500 PC 1130024605
span - POS 2400 50 PC 1120001286
span - POS 2700 500 PC 1120021038
span - POS 2800 50 PC 1120021657
span - POS 2900 200 PC 1120021686
span - POS 3000 75 PC 1120002537
span - POS 3100 950 PC 1120002541
span - POS 3200 1.400 PC 1120001167
span - 

In [67]:
print(fTextName.replace('.txt', '.py'))

C:\Users\alex1c\sources\spacySt\OCR\0093012939.py


In [ ]:
%run -i "C:\Users\alex1c\sources\spacySt\OCR\0093012934.py"

In [27]:
print(doc1)

Order number: 2378632 Purchase order number: N SR-1-06 1772 2400 25 PC 1120002537 38,09 100 PC 9,52 SP-2D-M-W2 SP 2 DM W2 packed per each item Product description: weld plate Export - Customs tariff no.: 73269098 Country of origin: Germany Material Surcharge 11,40 % 1,09 10,61 Order number


In [95]:
fTextName

'F:\\Python\\OCR_diploma\\docs_base\\StauffInvoices\\92946889.txt'

In [ ]:
def partCodePrepare(res, pos):
    pos = 1
    sDoc = 'doc' +str(pos)
    codeCom = '#'
    #code = sDoc+'.ents = [\n'
    firstPos = 100000
    lastPos = 0
    codeMain = ''
    for key in res.keys():
        lastPos = max(res.get(key)[1], lastPos)
        firstPos = min(firstPos, res.get(key)[0])
        codeCom += key + ' ' + res.get(key)[2].text + '; '
        if codeMain !='':
            codeMain += ',\n   span(' + sDoc + ', ' + str(res.get(key)[0]) + ', ' + str(res.get(key)[1]) + ', label ="' + key + '")'
        else:
            codeMain += '   span(' + sDoc + ', ' + str(res.get(key)[0]) + ', ' + str(res.get(key)[1]) + ', label ="' + key + '")'
    code = 'print("' + sDoc + ', '+ codeCom + '")\n' + sDoc + " = nlp('''" + doc1[firstPos: lastPos + 1].text + "''')\n"
    code += sDoc+'.ents = [\n' + codeMain + ']\ndocs.append(' + sDoc + ')\n'
    #print(codeCom, firstPos, lastPos)
    return code

print(partCodePrepare(res, 0))

In [53]:
with open(fName.replace('.txt', 's.txt'), encoding="utf8") as f:
    rtest = f.read()
rtest = rtest.replace(strSeparator, '')

nlp = spacy.blank("en")
matcher = Matcher(nlp.vocab)

patternPurchase = [{'LOWER': 'purchase'}, {'LOWER':'order'}, {'LOWER':'number'}, {'LOWER': ':'}]
matcher.add('PTRPURCHASE', [patternPurchase])

patternTariff = [{'LOWER': 'customs'}, {'LOWER':'tariff'}, {'IS_ALPHA': True}, {'IS_PUNCT': True, 'OP': '*'}, 
    {'IS_DIGIT': True}]
matcher.add('PTRTARIFF', [patternTariff])

patternCountry = [{'LOWER': 'country'}, {'LOWER':'of'}, {'LOWER': 'origin'}, {'LOWER': ':'}, {'IS_ALPHA': True}]
matcher.add('PTRCOUNTRY', [patternCountry])

patternPC = [{'LIKE_NUM': True}, {'LIKE_NUM': True}, {'LOWER' : {'in' : ['pc', 'рс']}}, {'LIKE_NUM':True}] 

matcher.add('PTROFPC', [patternPC])

patternSurcharge = [{'LOWER': 'material'}, {'LOWER': 'surcharge'}, {'LIKE_NUM' : True}, {'TEXT':'%'}, {'LIKE_NUM' : True},
    {'LIKE_NUM':True}    ] 

matcher.add('PTRSURCH', [patternSurcharge])

doc1 = nlp(rtest)

matches = matcher(doc1)
spans = [Span(doc1, start, end, label=match_id) for match_id, start, end in matches]

startSpan = 0
endSpan = 0
posTariff = 0
txt2 = 'import json \nimport spacy \nfrom spacy.matcher import Matcher \nfrom spacy.tokens import Span, DocBin \ndocs = [] \n'
nPos = 0
pos1PC = 0
pos2PC = 0
for sp in spans:
    if sp.label_  == 'PTRPURCHASE':
        startSpan = sp.start
        #print('here is a purchase', sp.start, doc1[sp.start: sp.end]) 
    elif sp.label_  == 'PTRTARIFF':
        posTariff = sp.end-1
        #print('here is a tariff', sp.end, doc1[sp.end-1: sp.end]) 
    elif sp.label_  == 'PTROFPC':
        pos2PC = sp.start
        #print('here is a summa', sp.end, doc1[sp.start: sp.end]) 

    elif sp.label_  == 'PTRCOUNTRY':
        endSpan = sp.end
        pos2PC = pos2PC - startSpan  
        #print(doc1[startSpan + pos2PC], doc1[startSpan + pos2PC +1 ],doc1[startSpan + pos2PC+2],doc1[startSpan + pos2PC+3],            doc1[startSpan + pos2PC+4])  
        #break
        #print(doc1[startSpan+pos2PC + 4].text )
        sDoc = "doc" + str(nPos)
        txt2 = txt2 + '\n\n' + sDoc + " = nlp('''" + doc1[startSpan:endSpan].text + "''')\n" 
        #txt2 = ''
        txt2 = txt2 + ('#' + sDoc + str(doc1[startSpan + 5 : startSpan + 8].text + ' '
            + doc1[startSpan + 8].text + ' ' + doc1[startSpan + 10].text + ' ' + doc1[startSpan + 11].text + ' '
            + doc1[startSpan+13].text + ' ' + doc1[startSpan+pos2PC].text + ' ' + doc1[startSpan+pos2PC + 1].text  + ' '
            + doc1[startSpan+pos2PC + 4].text + ' ' +  doc1[posTariff].text + ' ' + doc1[endSpan - 1].text).replace('\n', '') + '\n')
        #print(txt2)
        #break
        #txt2 = txt2 + sDoc + ".ents = [Span(" + sDoc + ', 3, 4, label="GPE")] \n'
        txt2 = txt2 + '''print(''' + '"' + sDoc + '",' + sDoc + "[5: 8], " + sDoc + "[8: 9], " + sDoc + "[10: 11], " + sDoc +"[11: 12], " 
        txt2 = txt2 +  sDoc + "[13: 14], " + sDoc + "["+str(pos2PC) + ':' + str(pos2PC + 1) + "], "
        txt2 = txt2 +  sDoc + "[" +str(pos2PC+1) + ':' + str(pos2PC + 2) + "] ," + sDoc + "["+str(pos2PC + 3) + ':' + str(pos2PC + 4) + "], " + sDoc + "["+ str(posTariff - startSpan)+ ': ' + str(posTariff - startSpan + 1) + "], " + sDoc + "[" + str(endSpan - startSpan - 1)+ ': ' + str(endSpan - startSpan) + "]) \n"
        txt2 = txt2 +  sDoc + ".ents = [Span(" + sDoc + ', 5, 8, label="CONTRACT"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', 8, 9, label="CONTRACT1"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', 10, 11, label="POS"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', 11, 12, label="AMOUNT"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', 13, 14, label="ARTICLE"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(pos2PC) + ', ' + str(pos2PC + 1) + ', label="PRICE"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(pos2PC + 1) + ', ' + str(pos2PC + 2) + ', label="UNIT"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(pos2PC + 3) + ', ' + str(pos2PC + 4) + ', label="SUM"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(posTariff - startSpan)+ ', ' + str(posTariff - startSpan + 1) + ', label="TARIFF"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(endSpan - startSpan - 1)+ ', ' + str(endSpan - startSpan) + ', label="COUNTRY")] \n'

        txt2 = txt2 + 'docs.append(' + sDoc + ')\n'
        nPos += 1
        #break
#print(txt2)
#exit

txt2 = txt2 + 'doc_bin = DocBin(docs = docs)\n'
txt2 = txt2 + "doc_bin.to_disk('./train.spacy')"
with open(fName.replace('.txt', '.py'), encoding="utf8", mode='w') as f:
    f.write(txt2)

In [66]:
%run -i "F:\Python\OCR_diploma\docs_base\StauffInvoices\92922860.py"
#%run -i "92922860.py"

SyntaxError: invalid syntax (92922860.py, line 41)

In [69]:
import json 
import spacy 
from spacy.matcher import Matcher 
from spacy.tokens import Span, DocBin 
docs = [] 


doc0 = nlp('''Purchase order number: N SR-1-06 942
1600 481 PC 6100076765 (*) 3,85 1 РС 1.851,85

ОКС-НР-12-Е-М201К-В-М/З
HP10-1-X0041N

packed per each item

Customer ID-No.: 6100076765
Product description: coupling

Export - Customs tariff no.: 84812010
Country of origin: Germany



Description
Material Surcharge 6,90 % 127,78''')
#doc0 c:SR-1-06 c1:942 pos:1600 a:481 art:PC pr:6100076765 u:( s:) trf:84812010 cnt: chr:Surcharge chrS:% TTL:
print("doc0", doc0[5: 8], doc0[8: 9], doc0[10: 11], doc0[11: 12], doc0[13: 14], doc0[17:18], doc0[18:19] ,doc0[20:21], doc0[623: 624], doc0[1184: 632], doc0[634: 635], doc0[636: 637], doc0[638: 639]  )

doc0.ents = [Span(doc0, 5, 8, label="CONTRACT"), 
    Span(doc0, 8, 9, label="CONTRACT1"), 
    Span(doc0, 10, 11, label="POS"), 
    Span(doc0, 11, 12, label="AMOUNT"), 
    Span(doc0, 568, 569, label="ARTICLE"), 
    Span(doc0, 17, 18, label="PRICE"), 
    Span(doc0, 18, 19, label="UNIT"), 
    Span(doc0, 20, 21, label="SUM"), 
    Span(doc0, 623, 624, label="TARIFF"), 
    Span(doc0, 1184, 632, label="COUNTRY"), 
    Span(doc0, 634, 635, label="CHARGE"), 
    Span(doc0, 636, 637, label="CHARGES"), 
    Span(doc0, 638, 639, label="TOTAL") ]
docs.append(doc0)



doc0 SR-1-06 942 1600 481 6100076765 3,85 1 1.851,85     


IndexError: [E035] Error creating span with start 568 and end 569 for Doc of length 82.

In [70]:
! python -m spacy init config ./config.cfg --lang en --pipeline ner


✘ The provided output file already exists. To force overwriting the config file,
set the --force or -F flag.



In [71]:
!python -m spacy train ./config.cfg --output ./output12 --paths.train train12.spacy --paths.dev dev12.spacy

^C
ℹ Saving to output directory: output12
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     45.36    0.00    0.00    0.00    0.00
  0     200         50.55   1420.67   99.70   99.70   99.70    1.00
  1     400          9.20     22.12  100.00  100.00  100.00    1.00
  3     600          2.37      2.65  100.00  100.00  100.00    1.00
  5     800          0.40      0.43  100.00  100.00  100.00    1.00
  8    1000          0.00      0.00  100.00  100.00  100.00    1.00
 11    1200          0.00      0.00  100.00  100.00  100.00    1.00
 15    1400          0.00      0.00  100.00  100.00  100.00    1.00
 20    1600          0.00 

[2022-05-16 14:31:14,453] [INFO] Set up nlp object from config
[2022-05-16 14:31:14,491] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-16 14:31:14,504] [INFO] Created vocabulary
[2022-05-16 14:31:14,507] [INFO] Finished initializing nlp object
[2022-05-16 14:31:16,723] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [ ]:
python -m spacy train ./config.cfg --output ./output --paths.train train.spacy --paths.dev dev.spacy

In [37]:
import random

d = [1,2,3,4,5,6,7]

random.shuffle(d)
print(d)

[7, 6, 3, 4, 2, 1, 5]


In [64]:
import json 
import spacy 
from spacy.matcher import Matcher 
from spacy.tokens import Span, DocBin 
docs = [] 


doc0 = nlp('''Purchase order number: N SR-1-06 604
1200 4 РС 6100074594 (*) 13,79 1 РС 55,16

QRC-FF-19-M-22L-BP-W3

FF19-2-L2230

packed per each item

Export - Customs tariff no.: 84812010
Country of origin: Germany''')
#doc0SR-1-06 604 1200 4 6100074594 13,79 1 РС 84812010 Germany
print("doc0",doc0[5: 8], doc0[8: 9], doc0[10: 11], doc0[11: 12], doc0[13: 14], doc0[17:18], doc0[18:19] ,doc0[19:20], doc0[50: 51], doc0[56: 57]) 
doc0.ents = [Span(doc0, 5, 8, label="CONTRACT"), 
    Span(doc0, 8, 9, label="CONTRACT1"), 
    Span(doc0, 10, 11, label="POS"), 
    Span(doc0, 11, 12, label="AMOUNT"), 
    Span(doc0, 13, 14, label="ARTICLE"), 
    Span(doc0, 17, 18, label="PRICE"), 
    Span(doc0, 18, 19, label="UNIT"), 
    Span(doc0, 19, 20, label="SUM"), 
    Span(doc0, 50, 51, label="TARIFF"), 
    Span(doc0, 56, 57, label="COUNTRY")] 
docs.append(doc0)


doc1 = nlp('''Purchase order number: N SR-1-06 711
1300 50 PC 6010003476 (*) 237,81 100 PC 118,91

FI-EWD-15L-V-W3-DKO
FI-EWD-15L-B-W3-DKO

packed per each item

Product description: fitting

Export - Customs tariff no.: 73079910
Country of origin: Germany''')
#doc1SR-1-06 711 1300 50 6010003476 237,81 100 PC 73079910 Germany
print("doc1",doc1[5: 8], doc1[8: 9], doc1[10: 11], doc1[11: 12], doc1[13: 14], doc1[17:18], doc1[18:19] ,doc1[19:20], doc1[59: 60], doc1[65: 66]) 
doc1.ents = [Span(doc1, 5, 8, label="CONTRACT"), 
    Span(doc1, 8, 9, label="CONTRACT1"), 
    Span(doc1, 10, 11, label="POS"), 
    Span(doc1, 11, 12, label="AMOUNT"), 
    Span(doc1, 13, 14, label="ARTICLE"), 
    Span(doc1, 17, 18, label="PRICE"), 
    Span(doc1, 18, 19, label="UNIT"), 
    Span(doc1, 19, 20, label="SUM"), 
    Span(doc1, 59, 60, label="TARIFF"), 
    Span(doc1, 65, 66, label="COUNTRY")] 
docs.append(doc1)


#print(doc1)


doc0 SR-1-06 604 1200 4 6100074594 13,79 1 РС 84812010 Germany
doc1 SR-1-06 711 1300 50 6010003476 237,81 100 PC 73079910 Germany


In [24]:
import json 
import spacy 
from spacy.matcher import Matcher 
from spacy.tokens import Span, DocBin 
docs = [] 


doc0 = nlp('''Purchase order number: N SR-1-06 604
1200 4 РС 6100074594 (*) 13,79 1 РС 55,16

QRC-FF-19-M-22L-BP-W3

FF19-2-L2230

packed per each item

Export - Customs tariff no.: 84812010
Country of origin: Germany''')
#doc0SR-1-06 604 1200 4 6100074594 13,79 1 РС 84812010 Germany
print("doc0",doc0[5: 8], doc0[8: 9], doc0[10: 11], doc0[11: 12], doc0[13: 14], doc0[17:18], doc0[18:19] ,doc0[19:20], doc0[50: 51], doc0[56: 57]) 
doc0.ents = [Span(doc0, 5, 8, label="CONTRACT"), 
    Span(doc0, 8, 9, label="CONTRACT1"), 
    Span(doc0, 10, 11, label="POS"), 
    Span(doc0, 11, 12, label="AMOUNT"), 
    Span(doc0, 13, 14, label="ARTICLE"), 
    Span(doc0, 17, 18, label="PRICE"), 
    Span(doc0, 18, 19, label="UNIT"), 
    Span(doc0, 19, 20, label="SUM"), 
    Span(doc0, 50, 51, label="TARIFF"), 
    Span(doc0, 56, 57, label="COUNTRY")] 
docs.append[doc0]


doc1 = nlp('''Purchase order number: N SR-1-06 711
1300 50 PC 6010003476 (*) 237,81 100 PC 118,91

FI-EWD-15L-V-W3-DKO
FI-EWD-15L-B-W3-DKO

packed per each item

Product description: fitting

Export - Customs tariff no.: 73079910
Country of origin: Germany''')
#doc1SR-1-06 711 1300 50 6010003476 237,81 100 PC 73079910 Germany
print("doc1",doc1[5: 8], doc1[8: 9], doc1[10: 11], doc1[11: 12], doc1[13: 14], doc1[17:18], doc1[18:19] ,doc1[19:20], doc1[59: 60], doc1[65: 66]) 
doc1.ents = [Span(doc1, 5, 8, label="CONTRACT"), 
    Span(doc1, 8, 9, label="CONTRACT1"), 
    Span(doc1, 10, 11, label="POS"), 
    Span(doc1, 11, 12, label="AMOUNT"), 
    Span(doc1, 13, 14, label="ARTICLE"), 
    Span(doc1, 17, 18, label="PRICE"), 
    Span(doc1, 18, 19, label="UNIT"), 
    Span(doc1, 19, 20, label="SUM"), 
    Span(doc1, 59, 60, label="TARIFF"), 
    Span(doc1, 65, 66, label="COUNTRY")] 
docs.append[doc1]


SR-1-06 604 1200 4 6100074594 13,79 1 55,16 84812010 Germany


In [61]:
pageinfo = {'ABOUTNOPAGE':0, 'STARTPAGE':0, 'ENDPAGE':0}
pageinfo['ABOUTNOPAGE'] = 10

print(pageinfo)

a = dict()
a['21'] = 10
a['22'] =100
print(a)

{'ABOUTNOPAGE': 10, 'STARTPAGE': 0, 'ENDPAGE': 0}
{'21': 10, '22': 100}


In [ ]:
doc1 = nlp('Purchase order number: 6100074594 ( * )')
doc1.ents = [Span(doc1, 4, 5, label='Article')]

doc2 = nlp('Country of origin: Germany')
doc2.ents = [Span(doc1, 4, 5, label='Country')]

In [ ]:
startPos = 483
for i, token in enumerate(doc[startPos: startPos + 300]):
    print(token)

In [41]:
doc1.text

'Purchase order number: N SR-1-06 711\n1300 50 PC 6010003476 (*) 237,81 100 PC 118,91\n\nFI-EWD-15L-V-W3-DKO\nFI-EWD-15L-B-W3-DKO\n\npacked per each item\n\nProduct description: fitting\n\nExport - Customs tariff no.: 73079910\nCountry of origin: Germany'

In [ ]:
rtest

In [54]:
nlp_ = spacy.load(r"C:\Users\alex1c\sources\spacySt\output12\model-best")
docx = nlp_(rtest.replace('. ', ' '))
for ent in docx.ents:
    print(ent.text, ent.label_)

2390568 NORDER
SR-1-06 CONTRACT
1911 CONTRACT1
17300 POS
13 AMOUNT
6020000130 ARTICLE
251,01 PRICE
100 UNIT
32,63 SUMMA
73079290 TARIFF
Germany COUNTRY
6,90 PR_SURCH
2,25 SURCHARGE
3,50 PR_ESURCH
1,14 ESURCHARGE
36,02 TOTAL
